# Text generation with an RNN

Generate text using a character-based RNN. Given a sequence of characters from this data, train a model to predict the next character in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.


## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download the dataset

Change the following line to run this code on your own data.'/content/drive/MyDrive/Development/Text generation

In [3]:
path_to_file = '/content/drive/MyDrive/Development/Text generation/towards_data_science_sentences.txt'

### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 60312847 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

Once upon a time , there were a guy and a girl . They had a date in Sydney Opera House . She was late as usual — something unexpected had happened — she couldn’t immediately find one of her 100 makeup products .  ( Other times she couldn’t find the o


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

1269 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[68, 69, 70, 71, 72, 73, 74], [91, 92, 93]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(60312847,), dtype=int64, numpy=array([50, 81, 70, ..., 85,  4, 17])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

O
n
c
e
 
u
p
o
n
 


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'O' b'n' b'c' b'e' b' ' b'u' b'p' b'o' b'n' b' ' b'a' b' ' b't' b'i'
 b'm' b'e' b' ' b',' b' ' b't' b'h' b'e' b'r' b'e' b' ' b'w' b'e' b'r'
 b'e' b' ' b'a' b' ' b'g' b'u' b'y' b' ' b'a' b'n' b'd' b' ' b'a' b' '
 b'g' b'i' b'r' b'l' b' ' b'.' b' ' b'T' b'h' b'e' b'y' b' ' b'h' b'a'
 b'd' b' ' b'a' b' ' b'd' b'a' b't' b'e' b' ' b'i' b'n' b' ' b'S' b'y'
 b'd' b'n' b'e' b'y' b' ' b'O' b'p' b'e' b'r' b'a' b' ' b'H' b'o' b'u'
 b's' b'e' b' ' b'.' b' ' b'S' b'h' b'e' b' ' b'w' b'a' b's' b' ' b'l'
 b'a' b't' b'e'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Once upon a time , there were a guy and a girl . They had a date in Sydney Opera House . She was late'
b' as usual \xe2\x80\x94 something unexpected had happened \xe2\x80\x94 she couldn\xe2\x80\x99t immediately find one of her 100 makeup p'
b'roducts .  ( Other times she couldn\xe2\x80\x99t find the other ones , so it was always different with the same '
b'result . ) While he was waiting , he googled fun facts about the Opera to impress her . This was what'
b' he had found on Wikipedia :\xe2\x80\x9cThat\xe2\x80\x99s so cool !\xe2\x80\x9d he thought . She said the same after hearing it . They'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Once upon a time , there were a guy and a girl . They had a date in Sydney Opera House . She was lat'
Target: b'nce upon a time , there were a guy and a girl . They had a date in Sydney Opera House . She was late'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 1270) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  325120    
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  1301750   
                                                                 
Total params: 5,565,174
Trainable params: 5,565,174
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([ 364,  664,  651,  992, 1256,  508, 1014,  639,  217,  595,  340,
        813, 1087,   92,  630,  985, 1044, 1145,  499,  604,  425,  154,
        847,  797,  912, 1133, 1141,   28,  456,  396, 1070,  845,  712,
       1117,  945,  349,  824,  386, 1169,  607,  409,  943,    4, 1157,
        387,  415,  140,  933,  728,  411,  109,  627,   32, 1035, 1026,
       1071, 1012,  146,   25,   96,  953,  439,  633,  694,   87, 1187,
       1254,  711,  152,  625,  806, 1255,  177,  363,  276,  827, 1231,
        451,  443, 1038,   90,  563,  867, 1162,  767,  549,  283,  642,
        159, 1142,  119, 1027,  666, 1212,   28,  565,  154,  396,  911,
        725])

Decode these to see the text predicted by this untrained model:

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'nneeded churn . The churn can either be tied to the feature or how it\xe2\x80\x99s communicated .Did users who '

Next Char Predictions:
 b'\xe2\x80\xa6\xe5\x90\x8d\xe5\x8a\x9b\xf0\x9f\x87\xb9\xf0\x9f\xa5\x87\xe2\x91\xa3\xf0\x9f\x8d\x8e\xe5\x83\x8d\xca\xb0\xe3\x81\x97\xe1\xb5\xa7\xea\xb3\xb5\xf0\x9f\x91\xaby\xe4\xbb\x8e\xf0\x9f\x85\xa1\xf0\x9f\x8f\x84\xf0\x9f\x94\x91\xe2\x8c\x8a\xe3\x82\xa4\xe2\x84\xa4\xc3\xb1\xef\xb8\xbd\xe9\x92\x9f\xf0\x9d\x92\xaf\xf0\x9f\x93\xb8\xf0\x9f\x94\x8d9\xe2\x88\x97\xe2\x82\x8a\xf0\x9f\x91\x81\xef\xac\x83\xe6\x88\xbf\xf0\x9f\x93\x89\xf0\x9d\x9b\x8c\xe2\x80\x90\xec\x84\xb8\xe2\x82\x80\xf0\x9f\x98\x82\xe3\x82\xaa\xe2\x82\x9a\xf0\x9d\x9b\x86 \xf0\x9f\x95\xb3\xe2\x82\x81\xe2\x84\x8e\xc3\xa3\xf0\x9d\x95\x90\xe6\x98\xaf\xe2\x82\x9c\xc2\xad\xe4\xba\x92=\xf0\x9f\x8e\xa8\xf0\x9f\x8e\x84\xf0\x9f\x91\x86\xf0\x9f\x8d\x8b\xc3\xa96}\xf0\x9d\x9b\xbe\xe2\x87\x94\xe4\xbd\x8d\xe5\xb1\x95t\xf0\x9f\x98\x94\xf0\x9f\xa4\xb8\xe6\x88\xa6\xc3\xaf\xe4\xba\x89\xe9\xbb\x83\xf

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 1270)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(7.1465807, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [36]:
tf.exp(example_batch_mean_loss).numpy()

1269.7568

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [37]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [39]:
EPOCHS = 2

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
9330/9330 [==============================] - 623s 66ms/step - loss: 1.3137
Epoch 2/2
9330/9330 [==============================] - 627s 67ms/step - loss: 1.1305


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [49]:
start = time.time()
states = None
next_char = tf.constant(['Not everyone who boarded the Titanic survived. If you need to predict whether a passenger survived or died, you can use the following model.'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Not everyone who boarded the Titanic survived. If you need to predict whether a passenger survived or died, you can use the following model. Data :So , who can do is at something discovered in them from balyn datasets , others would be indeed by blog .On each graph , you can try to add the data processing over your dataset to each section or building vertical data !To add-typically deployed iterations , -purpose question is foundation , both map showing the analysis model as a JupyterLab , it tures happen . Price Yetworks approach specifies 1×8 )  we always addressed 11OK to mirror of one of the models to compare messages off and testing the agion partitions between , single event of a 95% of 54 data , the relationship between the 1450 or 2D integrolling , and the output between our business values for each data . E or Mode , the Cloud provides software development that Cairs had both different epochs of tasks  ( W1 and Z new HTMult )  — and is a frame inside the training data form of 

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [50]:
start = time.time()
states = None
next_char = tf.constant(['Today', 'Python', 'NLP', 'SQL', 'Pandas'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:166x5d304In other examples page OMR , the latter , which is through the test table , we need to clance the final mechanism :The method depends on OPL conditioning and num approcial online science accessibility may fnet exaches things throughout death dropdowns  ( As-engineer\xe2\x80\x99s model . A quick tool it can be requaled , we will result in a new format as looking at the paper are positive and beep into them . At the right side , something lot of about a risk , so you may have had 82 part of a computer schems .Therefore , substituting these gridrences and every standard node using RIST , make originally published at 99 . Origin test data  ( 0 .79 64% )  . As a 5x3 ses , I offened SquaH\xc3\xa5kis\xe2\x80\x99 SVIDEET RDE method provides an idea as IozenRSAN did very small decent , biochine in data scientist . Naturally , because most of the data wants having results available , of course but unstacking the equations based on the task before supearth and all mid

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [55]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/Development/Text generation/model')
one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/Development/Text generation/model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Development/Text generation/model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Development/Text generation/model/assets


In [56]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:SEECA Hastep generally modelling . We can check whether were important for general probability  ( ac


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [57]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [58]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [59]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [60]:
model.fit(dataset, epochs=1)

9330/9330 [==============================] - 624s 66ms/step - loss: 1.3065


In [61]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['Not everyone who boarded the Titanic survived. If you need to predict whether a passenger survived or died, you can use the following model.'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Not everyone who boarded the Titanic survived. If you need to predict whether a passenger survived or died, you can use the following model. spow-based output .With this article , there pretrics don’t do is to understand all of other ways is becoming markdown uses of why it allows us to predict the following topics .Now we set the best power of hypothesis to check it !I look at the ApribSticy — yon’t see what’s based on the text data . In this article , we’re other components from skiftings from it and build columns of a confection of years each class . The size from the one coveriny this way was should sum quick exactly why you have seen however , let me know how we plot the specific taxes with the Key , Culturable about difference in your tracks person/GGD . Classification systems day a dataset  ( 1955N’C )  prevent Reduce — this was from each topic , using New24 ! This weight is a binary by Z in the original pseudo-labelpower (  )  , as a wime analysis of bandling of data-says that 

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10 

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))